In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:15 http://archive.ubuntu.com/ub

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/LDS9_K265_DinhVietHa/LDS9_K265_DinhVietHa_Cuoi_ky/'

/content/gdrive/My Drive/LDS9_K265_DinhVietHa/LDS9_K265_DinhVietHa_Cuoi_ky


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [ ]:
spark = SparkSession.builder.appName("Bakery").getOrCreate()

In [ ]:
bakery = spark.read.csv("./Du lieu cung cap/75000/75000i.csv", header=None,inferSchema=True)

In [ ]:
bakery.show(10)

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
|  1|  1| 21|
|  1|  5| 11|
|  2|  1|  7|
|  2|  3| 11|
|  2|  4| 37|
|  2|  3| 45|
|  3|  5|  3|
|  3|  3| 42|
|  3|  3| 33|
|  4|  1|  5|
+---+---+---+
only showing top 10 rows



In [ ]:
bakery = bakery.withColumnRenamed("_c0","order_ID")
bakery = bakery.withColumnRenamed("_c1","quantity")
bakery = bakery.withColumnRenamed("_c2","product_id")

In [ ]:
bakery.show(10)

+--------+--------+----------+
|order_ID|quantity|product_id|
+--------+--------+----------+
|       1|       1|        21|
|       1|       5|        11|
|       2|       1|         7|
|       2|       3|        11|
|       2|       4|        37|
|       2|       3|        45|
|       3|       5|         3|
|       3|       3|        42|
|       3|       3|        33|
|       4|       1|         5|
+--------+--------+----------+
only showing top 10 rows



In [ ]:
bakery.printSchema()

root
 |-- order_ID: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- product_id: integer (nullable = true)



In [ ]:
bakery.createOrReplaceTempView("order_products")

### Preprocess the data

In [ ]:
# preprocessing data
from pyspark.sql.functions import collect_list, col, count, collect_set, collect_list, struct

In [ ]:
products = spark.sql("select distinct product_id from order_products")
products.count()

50

In [ ]:
rawData = spark.sql("select * from order_products")
baskets = rawData.groupBy("order_id").agg(collect_set("product_id")\
                                          .alias('items'))
baskets.createOrReplaceTempView('baskets')

In [ ]:
baskets.show(5,truncate=False)

+--------+----------------------+
|order_id|items                 |
+--------+----------------------+
|148     |[33, 27, 9, 46, 28, 4]|
|463     |[17, 14]              |
|471     |[9, 37, 34, 20]       |
|496     |[15, 6, 47, 26]       |
|833     |[12, 5, 21]           |
+--------+----------------------+
only showing top 5 rows



In [ ]:
type(baskets)

pyspark.sql.dataframe.DataFrame

### Use flavor and food name instead of Id

In [ ]:
product_data = spark.read.csv("./Du lieu cung cap/75000/goods.csv",header=True,inferSchema=True)

In [ ]:
product_data.show(5)

+---+------------+------+-----+------+
| Id|      Flavor|  Food|Price|  Type|
+---+------------+------+-----+------+
|  0| 'Chocolate'|'Cake'| 8.95|'Food'|
|  1|     'Lemon'|'Cake'| 8.95|'Food'|
|  2|    'Casino'|'Cake'|15.95|'Food'|
|  3|     'Opera'|'Cake'|15.95|'Food'|
|  4|'Strawberry'|'Cake'|11.95|'Food'|
+---+------------+------+-----+------+
only showing top 5 rows



In [ ]:
product_data.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Flavor: string (nullable = true)
 |-- Food: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Type: string (nullable = true)



In [ ]:
#product_data_n = product_data.withColumn("text_new", concat(col("title"), lit(" "), col("text")).alias("text"))

In [ ]:
product_data.createOrReplaceTempView("products")

In [ ]:
rawData_1 = spark.sql('''select p.Flavor,p.Food, o.order_id from products p
                          inner join order_products o
                          where o.product_id = p.id''')
baskets_1 = rawData_1.groupBy('order_id').agg(collect_set(struct('Flavor','Food'))\
                                              .alias('items'))
baskets_1.createOrReplaceTempView('baskets')

In [ ]:
baskets_1.head(3)

[Row(order_id=148, items=[Row(Flavor="'Napoleon'", Food="'Cake'"), Row(Flavor="'Cheese'", Food="'Croissant'"), Row(Flavor="'Tuile'", Food="'Cookie'"), Row(Flavor="'Strawberry'", Food="'Cake'"), Row(Flavor="'Chocolate'", Food="'Coffee'"), Row(Flavor="'Marzipan'", Food="'Cookie'")]),
 Row(order_id=463, items=[Row(Flavor="'Berry'", Food="'Tart'"), Row(Flavor="'Chocolate'", Food="'Tart'")]),
 Row(order_id=471, items=[Row(Flavor="'Chocolate'", Food="'Croissant'"), Row(Flavor="'Almond'", Food="'Twist'"), Row(Flavor="'Pecan'", Food="'Tart'"), Row(Flavor="'Napoleon'", Food="'Cake'")])]

## The Model

In [ ]:
from pyspark.ml.fpm import FPGrowth

In [ ]:
fpGrowth_1 = FPGrowth(itemsCol='items',
                      minSupport=0.003,
                      minConfidence=0.003)
model_1 = fpGrowth_1.fit(baskets_1)

In [ ]:
# Display frequent itemsets
model_1.freqItemsets.show(truncate=False)

+-----------------------------------------------+----+
|items                                          |freq|
+-----------------------------------------------+----+
|[['Coffee', 'Eclair']]                         |8193|
|[['Hot', 'Coffee']]                            |7700|
|[['Hot', 'Coffee'], ['Coffee', 'Eclair']]      |2367|
|[['Tuile', 'Cookie']]                          |7556|
|[['Tuile', 'Cookie'], ['Hot', 'Coffee']]       |387 |
|[['Tuile', 'Cookie'], ['Coffee', 'Eclair']]    |383 |
|[['Cherry', 'Tart']]                           |6987|
|[['Cherry', 'Tart'], ['Tuile', 'Cookie']]      |393 |
|[['Cherry', 'Tart'], ['Hot', 'Coffee']]        |318 |
|[['Cherry', 'Tart'], ['Coffee', 'Eclair']]     |321 |
|[['Strawberry', 'Cake']]                       |6948|
|[['Strawberry', 'Cake'], ['Tuile', 'Cookie']]  |465 |
|[['Strawberry', 'Cake'], ['Hot', 'Coffee']]    |372 |
|[['Strawberry', 'Cake'], ['Cherry', 'Tart']]   |402 |
|[['Strawberry', 'Cake'], ['Coffee', 'Eclair']] |378 |
|[['Aprico

Some frequent items which are ordered together:
* coffee and eclair
* strawberry cake and hot coffee
* apricot danish and cherry tart

--> coffee is usually ordered along with a pastry.

In [ ]:
mostPopularItemInABasket_1 = model_1.transform(baskets_1)

In [ ]:
mostPopularItemInABasket_1.head(3)

[Row(order_id=148, items=[Row(Flavor="'Napoleon'", Food="'Cake'"), Row(Flavor="'Cheese'", Food="'Croissant'"), Row(Flavor="'Tuile'", Food="'Cookie'"), Row(Flavor="'Strawberry'", Food="'Cake'"), Row(Flavor="'Chocolate'", Food="'Coffee'"), Row(Flavor="'Marzipan'", Food="'Cookie'")], prediction=[Row(Flavor="'Apricot'", Food="'Danish'"), Row(Flavor="'Gongolais'", Food="'Cookie'"), Row(Flavor="'Hot'", Food="'Coffee'"), Row(Flavor="'Cherry'", Food="'Tart'"), Row(Flavor="'Orange'", Food="'Juice'"), Row(Flavor="'Coffee'", Food="'Eclair'"), Row(Flavor="'Berry'", Food="'Tart'"), Row(Flavor="'Apricot'", Food="'Croissant'"), Row(Flavor="'Lemon'", Food="'Cake'"), Row(Flavor="'Chocolate'", Food="'Cake'"), Row(Flavor="'Blueberry'", Food="'Tart'"), Row(Flavor="'Truffle'", Food="'Cake'"), Row(Flavor="'Opera'", Food="'Cake'"), Row(Flavor="'Vanilla'", Food="'Frappuccino'"), Row(Flavor="'Almond'", Food="'Twist'"), Row(Flavor="'Apple'", Food="'Pie'"), Row(Flavor="'Blackberry'", Food="'Tart'"), Row(Flavor="

In [ ]:
type(mostPopularItemInABasket_1)

pyspark.sql.dataframe.DataFrame

In [ ]:
# convert list array to string
from pyspark.sql.types import StringType

In [ ]:
mostPopularItemInABasket_1.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Flavor: string (nullable = true)
 |    |    |-- Food: string (nullable = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Flavor: string (nullable = true)
 |    |    |-- Food: string (nullable = true)



In [ ]:
mostPopularItemInABasket_1.createOrReplaceTempView("popular_items")

In [ ]:
DF_cast = mostPopularItemInABasket_1.select("order_id",
                                            mostPopularItemInABasket_1.items.cast(StringType()),
                                            mostPopularItemInABasket_1.prediction.cast(StringType()))

In [ ]:
DF_cast.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- items: string (nullable = true)
 |-- prediction: string (nullable = true)



In [ ]:
DF_cast.head(3)

[Row(order_id=148, items="[['Napoleon', 'Cake'], ['Cheese', 'Croissant'], ['Tuile', 'Cookie'], ['Strawberry', 'Cake'], ['Chocolate', 'Coffee'], ['Marzipan', 'Cookie']]", prediction="[['Apricot', 'Danish'], ['Gongolais', 'Cookie'], ['Hot', 'Coffee'], ['Cherry', 'Tart'], ['Orange', 'Juice'], ['Coffee', 'Eclair'], ['Berry', 'Tart'], ['Apricot', 'Croissant'], ['Lemon', 'Cake'], ['Chocolate', 'Cake'], ['Blueberry', 'Tart'], ['Truffle', 'Cake'], ['Opera', 'Cake'], ['Vanilla', 'Frappuccino'], ['Almond', 'Twist'], ['Apple', 'Pie'], ['Blackberry', 'Tart'], ['Lemon', 'Tart'], ['Bottled', 'Water'], ['Casino', 'Cake'], ['Chocolate', 'Tart'], ['Lemon', 'Lemonade'], ['Apple', 'Tart'], ['Lemon', 'Cookie'], ['Single', 'Espresso'], ['Walnut', 'Cookie'], ['Raspberry', 'Lemonade'], ['Apple', 'Danish'], ['Raspberry', 'Cookie'], ['Apple', 'Croissant'], ['Green', 'Tea'], ['Cherry', 'Soda'], ['Blueberry', 'Danish'], ['Pecan', 'Tart'], ['Chocolate', 'Croissant'], ['Ganache', 'Cookie'], ['Almond', 'Croissant']